Import of Packages

In [2]:
import scipy.stats as stats
import heapq
from collections import defaultdict
#from Code.import_data import import_data
from datetime import datetime, timedelta
import pandas as pd

Import of Data

In [3]:

def import_data():
    # @diana and denis, change to your directory
    gtfs_dir = '/Users/paulinaheine/Codes/BackupTravelPlanning/GTFS_OP_2024_obb-2/'

    # Load data
    agency_df = pd.read_csv(gtfs_dir + 'agency.txt')
    stops_df = pd.read_csv(gtfs_dir + 'stops.txt')
    routes_df = pd.read_csv(gtfs_dir + 'routes.txt')
    trips_df = pd.read_csv(gtfs_dir + 'trips.txt')
    stop_times_df = pd.read_csv(gtfs_dir + 'stop_times.txt')
    calendar_df = pd.read_csv(gtfs_dir + 'calendar.txt')
    calendar_dates_df = pd.read_csv(gtfs_dir + 'calendar_dates.txt')
    shapes_df = pd.read_csv(gtfs_dir + 'shapes.txt')
    
    # Return all DataFrames
    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df, calendar_dates_df, shapes_df

# Import and display data


Helping Functions

In [4]:
def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 60 + minutes + seconds / 60

def minutes_to_time(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

def get_weekday(date):
    weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return weekdays[date.weekday()]

def is_service_available(service_id, start_time_obj, calendar, calendar_dates):
    #date_str = date.strftime("%Y%m%d")
    #date = date.date()
    # Convert date to string in YYYYMMDD format
    date_str = start_time_obj.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_time_obj.weekday()]

    # Step 1: Check for exceptions in calendar_dates
    if service_id in calendar_dates["service_id"].values:
        exceptions = calendar_dates[calendar_dates["service_id"] == service_id]
        for _, exception in exceptions.iterrows():
            if exception["date"] == int(date_str):
                if exception["exception_type"] == 2:  # Service is added as an exception
                    return True
                #elif exception["exception_type"] == 1:  # Service is removed as an exception
                #    return False

    # Step 2: Check for regular service in calendar
    if service_id in calendar["service_id"].values:
        service = calendar[calendar["service_id"] == service_id]
        # Check if date is within start_date and end_date
        if int(service["start_date"].iloc[0]) <= int(date_str) <= int(service["end_date"].iloc[0]):
            # Check if the service operates on this weekday
            is_available = (service[weekday].iloc[0] == 1)
            if is_available == 1:
                return True
    return False

def prepare_calendar_dates(calendar_dates):
    grouped = calendar_dates.groupby("service_id")
    calendar_dates_dict = {}
    for service_id, group in grouped:
        exceptions = group.to_dict(orient="records")
        calendar_dates_dict[service_id] = exceptions
    return calendar_dates_dict

In [6]:
agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()

Create Graph

In [7]:
def create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj):
    '''
    funktioniert endlich, if not wurde entfernt weil es mich genervt hat 3-4 min 
    '''

    graph = defaultdict(list)
    stop_id_to_name = stops.set_index("stop_id")["stop_name"].to_dict()
    # Filter for active trips today using is_service
    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    trip_id_to_route = trips.set_index("trip_id")["route_id"].to_dict()

    calendar_dates_2 = prepare_calendar_dates(calendar_dates)
    stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])

    #####
    #Filter
    #####

    stop_times_copy = stop_times.copy()
    # Filter out stops outside the time window
    stop_times_copy["arrival_minutes"] = stop_times_copy["arrival_time"].apply(time_to_minutes)
    stop_times_copy["departure_minutes"] = stop_times_copy["departure_time"].apply(time_to_minutes)

    start_minutes = time_to_minutes(start_time_obj.strftime("%H:%M:%S"))
    end_minutes = time_to_minutes(end_time_obj.strftime("%H:%M:%S"))

    stop_times = stop_times_copy[
        (stop_times_copy["arrival_minutes"] >= start_minutes) &
        (stop_times_copy["departure_minutes"] <= end_minutes)
        ]

    print(f"Rows after time window filter: {len(stop_times)}")


    #trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)

    grouped = stop_times.groupby("trip_id")
    for trip_id, group in grouped:
        service_id = trip_id_to_service.get(trip_id)
        
        if is_service_available(service_id, start_time_obj, calendar, calendar_dates) == True:
    
            stops = group["stop_id"].tolist()
            departures = group["departure_time"].apply(time_to_minutes).tolist()
            arrivals = group["arrival_time"].apply(time_to_minutes).tolist()
    
            for start, end, dep, arr in zip(stops[:-1], stops[1:], departures[:-1], arrivals[1:]):
                travel_time = arr - dep
                if travel_time > 0:
                    graph[stop_id_to_name[start]].append(
                        (stop_id_to_name[end], dep, arr, trip_id_to_route.get(trip_id))
                    )

    return graph


Reliability function

In [8]:
def compute_transfer_probability_with_departure_delay(transfer_time):
    #transfer_window = departure_time - scheduled_arrival
    #if transfer_window < 0:
    #    return 0.1  # No chance of a successful transfer if arrival is after departure
    return stats.gamma.cdf(transfer_time, a=2, scale=3)


Djikstra

In [102]:
def dijkstra_with_reliability_fixed(graph, start_name, end_name, start_time_minutes, time_budget_minutes,
                                    exclude_routes=set(), MIN_TRANSFER_TIME= 2):
    pq = [(start_time_minutes, start_name, [], 1.0,
           None)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie)
    visited = set()
    # Mindestumstiegszeit in Minuten

    while pq:
        current_time, current_stop, path, reliability, last_route = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]

        if current_time - start_time_minutes > time_budget_minutes:
            continue  # Pruning: Abbruch, wenn Zeitbudget überschritten

        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    # Mindestumstiegszeit von 5 Minuten nur bei Umstiegen
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Pruning: Zu wenig Zeit für Umstieg

                # Berechne neue Zuverlässigkeit mit Transferwahrscheinlichkeit
                if not is_transfer:  # Keine Zuverlässigkeitsänderung bei gleicher Linie
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                heapq.heappush(pq, (
                    new_current_time, neighbor, path + [(route_id, departure_time, arrival_time)], new_reliability,
                    route_id))

        if current_stop == end_name:
            return current_time, path, reliability

    return float("inf"), [], 0.0  # Keine Route gefunden

Backup Djikstra

In [10]:

def find_backup_routes(graph, primary_path, start_time_minutes, time_budget_minutes):
    backup_routes = []
    used_routes = {segment[0] for segment in primary_path if isinstance(segment, tuple) and len(segment) == 3}

    for i in range(1, len(primary_path) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, transfer_time = primary_path[i - 1]

        if transfer_stop in graph:
            backup_time, backup_path, backup_reliability = dijkstra_with_reliability_fixed(
                graph, transfer_stop, primary_path[-1][0], transfer_time, time_budget_minutes, used_routes
            )

            if backup_time < float("inf") and backup_path and backup_path != primary_path:
                alternative_path = []
                alternative_routes = set()

                for j in range(1, len(backup_path) - 1, 2):
                    stop1, time1 = backup_path[j - 1]
                    route, dep, arr = backup_path[j]
                    stop2, _ = backup_path[j + 1]

                    # **Kriterium für alternative Backup-Route:**
                    # - Die Route darf nicht exakt der Primärroute entsprechen.
                    # - Die Haltestellen dürfen wiederverwendet werden, aber nicht in identischen Abschnitten.
                    if route not in used_routes:
                        alternative_path.append((stop1, minutes_to_time(dep), stop2, minutes_to_time(arr), route))
                        alternative_routes.add(route)

                if alternative_path:
                    backup_routes.append((transfer_stop, alternative_path, backup_reliability))
                    used_routes.update(alternative_routes)  # Speichere die genutzten alternativen Linien

    return backup_routes  # Gibt jetzt echte alternative Backup-Routen zurück

# TODO Only exlude route before transfer and one after
# TODO search for mreliable
# TODO Backup rout would start at leats at departure time of missed connection



MAIN

In [11]:

# Hauptprogramm
if __name__ == "__main__":

    agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()

    start_stop_name = "Schattendorf Kirchengasse"
    end_stop_name = "Flughafen Wien Bahnhof"
    start_datetime = "2024-12-12 14:30:00"

    time_budget = "2:10"
    time_budget_hours, time_budget_minutes = map(int, time_budget.split(":"))
    time_budget_minutes = time_budget_hours * 60 + time_budget_minutes / 60

    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    # In ein reines date-Objekt umwandeln
    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    end_time_obj = start_time_obj + timedelta(minutes=time_budget_minutes)
    date_obj = start_time_obj.date()

    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    #fast and weird
    #graph = create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj)

    #slow and correct
    print("creating graph")
    graph = create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj)
    if start_stop_name not in graph or end_stop_name not in graph:
        print("🚨 Ungültige Start- oder Zielhaltestelle!")
        #sys.exit()

    # Haupt-Dijkstra-Lauf
    print("djikstra")
    arrival_time_minutes_fixed, path_fixed, reliability_fixed = dijkstra_with_reliability_fixed(
        graph, start_stop_name, end_stop_name, start_time_minutes, time_budget_minutes,MIN_TRANSFER_TIME=5
    )

    # Backup-Routen berechnen
    print("backups")
    backup_routes = find_backup_routes(graph, path_fixed, start_time_minutes, time_budget_minutes = 3000, MIN_TRANSFER_TIME= 10)

    # Ergebnis anzeigen
    if arrival_time_minutes_fixed < float("inf"):
        arrival_time_fixed = minutes_to_time(arrival_time_minutes_fixed)
        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")

        last_route = None
        grouped_routes = []

        for i in range(1, len(path_fixed) - 1, 2):
            current_stop, current_time = path_fixed[i - 1]
            route_id, departure_time, arrival_time = path_fixed[i]
            next_stop, _ = path_fixed[i + 1]

            if route_id == last_route:
                grouped_routes[-1]["stops"].append((next_stop, arrival_time))
            else:
                grouped_routes.append({
                    "route_id": route_id,
                    "start_stop": current_stop,
                    "departure_time": departure_time,
                    "stops": [(next_stop, arrival_time)]
                })
            last_route = route_id

        for segment in grouped_routes:
            start = segment["start_stop"]
            dep_time = minutes_to_time(segment["departure_time"])
            route = segment["route_id"]
            stops = " → ".join([f"{stop} (Ankunft: {minutes_to_time(arr)})" for stop, arr in segment["stops"]])
            print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

        print(f"\n🎯 Endstation: {end_stop_name} (Ankunft: {arrival_time_fixed})")
        print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability_fixed:.2f}\n")


        print("🔄 Backup-Routen:")
        if backup_routes:
            print("\n🔄 Backup-Routen:")
            for stop, path, reliability in backup_routes:
                print(f"  🔁 Backup von {stop}:")
                last_route = None
                first_segment = True

                for segment in path:
                    start, dep_time, end, arr_time, route = segment

                    if last_route == route:
                        print(f" → {end} (Ankunft: {arr_time})", end="")
                    else:
                        if not first_segment:
                            print()
                        print(f"    🚆 {start} (Abfahrt: {dep_time}) → {end} (Ankunft: {arr_time}) mit Linie {route}",
                              end="")
                        first_segment = False

                    last_route = route

                print(f"\n    🔹 Zuverlässigkeit: {reliability:.2f}\n")
        else:
            print("  ❌ Keine Backup-Routen verfügbar.")

        print()
    else:
        print(f"\n⚠️ Keine zuverlässige Route von {start_stop_name} nach {end_stop_name} gefunden.\n")



creating graph
Rows after time window filter: 32584


/var/folders/qw/jbl_sx2j4r114n9k1myblk6c0000gn/T/ipykernel_54612/2638892618.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)


djikstra
backups


TypeError: find_backup_routes() got an unexpected keyword argument 'MIN_TRANSFER_TIME'

In [96]:
def find_backup_routes(graph, path_fixed, start_time_minutes, time_budget_minutes=3000, MIN_TRANSFER_TIME=10):
    """
    Findet die zuverlässigsten Backup-Routen für jeden Umstiegspunkt in der Primärroute.

    :param graph: Der Transportgraph.
    :param path_fixed: Die Primärroute, berechnet durch Dijkstra.
    :param start_time_minutes: Startzeit in Minuten.
    :param time_budget_minutes: Zeitbudget in Minuten.
    :param MIN_TRANSFER_TIME: Minimale Transferzeit in Minuten.
    :return: Liste der zuverlässigsten Backup-Routen für jeden Umstiegspunkt.
    """
    # Berechne used_routes: Nur die ersten Legs nach jedem Transfer ausschließen
    used_routes = set()
    for i in range(1, len(path_fixed) - 1, 2):
        current_leg = path_fixed[i]
        next_leg = path_fixed[i + 1] if i + 1 < len(path_fixed) else None

        if next_leg and isinstance(next_leg, tuple) and next_leg[0] != current_leg[2]:
            used_routes.add(current_leg)

    # Sicherstellen, dass spezifische Legs ausgeschlossen werden
    specific_exclusions = {
        ('3-R93-A-j24-1', 894.0, 898.0),
        ('10-A12-j24-1', 930.0, 955.0),
        ('10-A3-j24-1', 969.0, 984.0)
    }
    used_routes.update(specific_exclusions)

    backup_routes = []
    last_route = None    

    for i in range(1, len(path_fixed) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, transfer_time = path_fixed[i - 1]
        
        adjusted_transfer_time = MIN_TRANSFER_TIME + 15
        current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]

        if route_id != last_route:
            # Berechne Backup-Route ab dem aktuellen Umstiegspunkt
            backup_time, backup_path, backup_reliability = dijkstra_with_reliability_fixed(
                graph, transfer_stop, path_fixed[-1][0], transfer_time, time_budget_minutes, used_routes
            )

            if backup_time < float("inf") and backup_path and backup_path != path_fixed:
                alternative_path = []
                alternative_routes = set()

                for j in range(1, len(backup_path) - 1, 2):
                    stop1, time1 = backup_path[j - 1]
                    route, dep, arr = backup_path[j]
                    stop2, _ = backup_path[j + 1]

                    # **Kriterium für alternative Backup-Route:**
                    if route not in used_routes:
                        alternative_path.append((stop1, minutes_to_time(dep), stop2, minutes_to_time(arr), route))
                        alternative_routes.add(route)

                if alternative_path:
                    backup_routes.append((transfer_stop, alternative_path, backup_reliability))
                    used_routes.update(alternative_routes)  # Speichere die genutzten alternativen Linien

            last_route = route_id
        
    return backup_routes



In [111]:
def find_backup_routes(graph, path_fixed, time_budget_minutes = 3000, MIN_TRANSFER_TIME = 15):
    """
    (graph, start_name, end_name, start_time_minutes, time_budget_minutes,
                                    exclude_routes=set()):
    Findet die zuverlässigste Backup-Route für jeden Umstiegspunkt in der Primärroute.

    :param graph: Der Transportgraph.
    :param primary_path: Die Primärroute, berechnet durch Dijkstra.
    :param start_time_minutes: Startzeit in Minuten.
    :param time_budget_minutes: Zeitbudget in Minuten.
    :param min_transfer_time: Minimale Transferzeit in Minuten.
    :return: Liste der zuverlässigsten Backup-Routen für jeden Umstiegspunkt.
    """
    backup_routes = []

    last_route = None    


        
    for i in range(1, len(path_fixed) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, transfer_time = path_fixed[i - 1]
    # Identifiziere alle Umstiegspunkte in der Route
   # for i in range(1, len(path_fixed) - 1, 2):
        
        #adjusted_transfer_time = MIN_TRANSFER_TIME + 15
        current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]

        if route_id != last_route:

        
            # Berechne Backup-Route ab dem aktuellen Umstiegspunkt

            backup_time, backup_path, backup_reliability = dijkstra_with_reliability_fixed(
                graph, transfer_stop, path_fixed[-1][0], current_time, time_budget_minutes,
                                    exclude_routes=set(), MIN_TRANSFER_TIME= 15
            )

            '''
            if backup_time < float("inf") and backup_path and backup_path !=path_fixed:
                alternative_path = []
                alternative_routes = set()

                for j in range(1, len(backup_path) - 1, 2):
                    stop1, time1 = backup_path[j - 1]
                    route, dep, arr = backup_path[j]
                    stop2, _ = backup_path[j + 1]

                    # **Kriterium für alternative Backup-Route:**
                    # - Die Route darf nicht exakt der Primärroute entsprechen.
                    # - Die Haltestellen dürfen wiederverwendet werden, aber nicht in identischen Abschnitten.
                    if route not in used_routes:
            '''
            
            alternative_path = []
            alternative_routes = set()
            for j in range(1, len(backup_path) - 1, 2):
                stop1, time1 = backup_path[j - 1]
                route, dep, arr = backup_path[j]
                stop2, _ = backup_path[j + 1]
                
                
                alternative_path.append((stop1, minutes_to_time(dep), stop2, minutes_to_time(arr), route))
                alternative_routes.add(route)

                if alternative_path:
                    backup_routes.append((transfer_stop, alternative_path, backup_reliability))
                    #used_routes.update(alternative_routes)  # Speichere die genutzten alternativen Linien

            


    
        last_route = route_id
        
    # Berechne Backup-Routen ab dem Umstiegspunkt


    return backup_routes





In [119]:
backup_routes = find_backup_routes(graph,path_fixed,  time_budget_minutes = 300, MIN_TRANSFER_TIME = 15)


In [120]:
backup_routes

[(inf, [], 0.0),
 (inf, [], 0.0),
 (987.0,
  [('Wiener Neustadt Hauptbahnhof', 925.0),
   ('10-A12-j24-1', 927.0, 955.0),
   ('Wien Meidling', 955.0),
   ('10-A12-j24-1', 957.0, 962.0),
   ('Wien Hauptbahnhof', 962.0),
   ('10-A12-j24-1', 972.0, 987.0),
   ('Flughafen Wien Bahnhof', 987.0)],
  1.0),
 (984.0,
  [('Wien Hauptbahnhof', 962.0),
   ('10-A3-j24-1', 969.0, 984.0),
   ('Flughafen Wien Bahnhof', 984.0)],
  1.0)]

In [128]:

    backup_routes = []

    last_route = None    

    for i in range(1, len(path_fixed) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, transfer_time = path_fixed[i - 1]
    # Identifiziere alle Umstiegspunkte in der Route
   # for i in range(1, len(path_fixed) - 1, 2):
        
        #adjusted_transfer_time = MIN_TRANSFER_TIME + 15
        current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]

        if route_id != last_route:
            print(current_stop, current_time,route_id, departure_time, arrival_time,next_stop)
        last_route = route_id    

Schattendorf Kirchengasse 870 20-SV5-L-j24-1 883.0 888.0 Loipersbach-Schattendorf Bhf.
Loipersbach-Schattendorf Bhf. 888.0 3-R93-A-j24-1 894.0 898.0 Marz-Rohrbach Bahnhof
Wiener Neustadt Hauptbahnhof 925.0 10-A12-j24-1 930.0 955.0 Wien Meidling
Wien Hauptbahnhof 962.0 10-A3-j24-1 969.0 984.0 Flughafen Wien Bahnhof


In [124]:
    backup_routes = []
    min_transfer_time = 5
    last_route = None
    alternative_routes = set()
    # Identifiziere alle Umstiegspunkte in der Route
    for i in range(1, len(path_fixed) - 1, 2):
        adjusted_transfer_time = min_transfer_time + 5
        current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]

        if route_id != last_route:
            #print(current_stop,route_id,last_route,next_stop,current_time)
                    
            backup_time, backup_path, backup_reliability = dijkstra_with_reliability_fixed(
                graph, transfer_stop, path_fixed[-1][0], current_time, time_budget_minutes,
                                    exclude_routes=set(), MIN_TRANSFER_TIME= 15)
            print(backup_time, backup_path, backup_reliability )
        
            alternative_path = [] 
            for j in range(1, len(backup_path) - 1, 2):
                stop1, time1 = backup_path[j - 1]
                route, dep, arr = backup_path[j]
                stop2, _ = backup_path[j + 1]
                
                
                alternative_path.append((stop1, minutes_to_time(dep), stop2, minutes_to_time(arr), route))
                alternative_routes.add(route)

                if alternative_path:
                    backup_routes.append((transfer_stop, alternative_path, backup_reliability))
            
        last_route = route_id
    print(backup_routes)


Schattendorf Kirchengasse 20-SV5-L-j24-1 None Loipersbach-Schattendorf Bhf. 870
897.0 [('Wien Hauptbahnhof', 870), ('10-A1-j24-1', 880.0, 897.0), ('Flughafen Wien Bahnhof', 897.0)] 1.0
Loipersbach-Schattendorf Bhf. 3-R93-A-j24-1 20-SV5-L-j24-1 Marz-Rohrbach Bahnhof 888.0
924.0 [('Wien Hauptbahnhof', 888.0), ('10-A11-j24-1', 907.0, 924.0), ('Flughafen Wien Bahnhof', 924.0)] 1.0
Wiener Neustadt Hauptbahnhof 10-A12-j24-1 3-R93-A-j24-1 Wien Meidling 925.0
957.0 [('Wien Hauptbahnhof', 925.0), ('10-A12-j24-1', 942.0, 957.0), ('Flughafen Wien Bahnhof', 957.0)] 1.0
Wien Hauptbahnhof 10-A3-j24-1 10-A12-j24-1 Flughafen Wien Bahnhof 962.0
984.0 [('Wien Hauptbahnhof', 962.0), ('10-A3-j24-1', 969.0, 984.0), ('Flughafen Wien Bahnhof', 984.0)] 1.0
[('Wien Hauptbahnhof', [('Wien Hauptbahnhof', '14:40', 'Flughafen Wien Bahnhof', '14:57', '10-A1-j24-1')], 1.0), ('Wien Hauptbahnhof', [('Wien Hauptbahnhof', '15:07', 'Flughafen Wien Bahnhof', '15:24', '10-A11-j24-1')], 1.0), ('Wien Hauptbahnhof', [('Wien H

In [126]:
current_time

962.0

In [117]:
path_fixed  

[('Schattendorf Kirchengasse', 870),
 ('20-SV5-L-j24-1', 883.0, 888.0),
 ('Loipersbach-Schattendorf Bhf.', 888.0),
 ('3-R93-A-j24-1', 894.0, 898.0),
 ('Marz-Rohrbach Bahnhof', 898.0),
 ('3-R93-A-j24-1', 899.0, 901.0),
 ('Mattersburg Bahnhof', 901.0),
 ('3-R93-A-j24-1', 902.0, 903.0),
 ('Mattersburg Nord Bahnhof', 903.0),
 ('3-R93-A-j24-1', 904.0, 907.0),
 ('Wiesen-Sigleß Bahnhof', 907.0),
 ('3-R93-A-j24-1', 907.0, 911.0),
 ('Bad Sauerbrunn Bahnhof', 911.0),
 ('3-R93-A-j24-1', 913.0, 916.0),
 ('Neudörfl Bahnhof', 916.0),
 ('3-R93-A-j24-1', 917.0, 920.0),
 ('Katzelsdorf Bahnhof', 920.0),
 ('3-R93-A-j24-1', 920.0, 925.0),
 ('Wiener Neustadt Hauptbahnhof', 925.0),
 ('10-A12-j24-1', 930.0, 955.0),
 ('Wien Meidling', 955.0),
 ('10-A12-j24-1', 957.0, 962.0),
 ('Wien Hauptbahnhof', 962.0),
 ('10-A3-j24-1', 969.0, 984.0),
 ('Flughafen Wien Bahnhof', 984.0)]